<a href="https://colab.research.google.com/github/Cherrise-exe/longFiles_watsonSpeechToText/blob/main/Speech_To_Text_Long_Files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Install and Import Dependencies

In [56]:
pip install ibm_watson

In [58]:
from ibm_watson import SpeechToTextV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

2. Setup STT Service

In [59]:
apikey = 'u9dE1TYnndD77H6uCTxPazqixP_Yv1Aj6Rz56Ui_tKbu'
url = 'https://api.us-south.speech-to-text.watson.cloud.ibm.com/instances/4313f2f0-ec19-4f25-812e-8e12ae0715f2'

In [60]:
authenticator = IAMAuthenticator(apikey)
stt = SpeechToTextV1(authenticator=authenticator)
stt.set_service_url(url)

3. Compress and Split Audio

In [61]:
import subprocess
import os
command = 'ffmpeg -i NASA_Space_Story_1074.wav -vn -ar 44100 -ac 2 -b:a 192k audio.mp3'
subprocess.call(command, shell=True)
command = 'ffmpeg -i audio.mp3 -f segment -segment_time 360 -c copy %03d.mp3'
subprocess.call(command, shell=True)

0

In [63]:
files = []
for filename in os.listdir('.'):
  if filename.endswith(".mp3") and filename != 'audio.mp3':
    files.append(filename)
files.sort()

In [64]:
files

['000.mp3', '001.mp3', '002.mp3', '003.mp3']

4. Perform Conversion

In [65]:
results = []
for filename in files:
  with open(filename, 'rb') as f:
    res = stt.recognize(audio=f, content_type='audio/mp3', model='en-US_NarrowbandModel', \
                        inactivity_timeout=360).get_result()
    results.append(res)               

In [67]:
text = []
for file in results:
  for result in file['results']:
    text.append(result['alternatives'][0]['transcript'].rstrip() + '.\n')

In [68]:
text

['they still the one thousand seventy four four minutes thirty seconds coming in three two one.\n',
 'on the fifteenth anniversary of Apollo eleven NASA for exams the space storage.\n',
 'may twenty fifth nineteen sixty one I believe that this nation should commit itself to achieving the goal before this decade is out of landing a man on the moon and returning him safely to the earth.\n',
 'seconds and counting.\n',
 'astronauts reported feels good two minutes twenty five seconds.\n',
 'twenty seconds and counting.\n',
 'T. minus fifteen seconds guidance is internal twelve eleven ten nine ignition sequence start.\n',
 'six five four.\n',
 '%HESITATION.\n',
 'engine.\n',
 "two years later on July sixteenth nineteen sixty nine Apollo eleven was hurled at his face from NASA's Kennedy Space Center Jerry in the crew three astronauts Neil Armstrong buzz Aldrin and Michael Collins.\n",
 "the great voyage to the moon was under way Roger eleven we'll pass that on and it looks like %HESITATION o

In [69]:
with open('output.txt', 'w') as out:
  out.writelines(text)